In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
#import string

import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
startyear = 2003
endyear   = 2008

In [ ]:
cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.deep_r')(np.linspace(0,.4,255)))
cmap2 = 'cmo.rain'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.2,.6,255)))

xx,y0,y1 = 3530,7900,8100

In [ ]:
vmax = 100
vmin = -10
linthresh=1
linscale=.2

vmax = 100
vmin = -10
linthresh = .3
linscale = .25


fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#print(fracpos)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,.9,nneg))
colors2 = plt.cm.inferno(np.linspace(0.1, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)

In [ ]:
plt.style.use('style_paper')

#mpl.rc('figure.subplot',left=.01,right=.98,top=.95,bottom=.15,wspace=.05,hspace=.2)
#mpl.rc('text',usetex=False)
fig = plt.figure(figsize=(7,4.5))

fig,ax = plt.subplots(2,3,figsize=(7,4),sharey='row',sharex=True)

for aa,xx in enumerate([3560,3536,3522]):

    #Geometry
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(xx,xx+1),y=slice(y0,y1))
    ds = add_lonlat(ds)

    xarr = .5*(len(ds.y)-np.arange(0,len(ds.y)))

    dax = ax[0,aa]
    #dax.plot(ds.lon,ds.surface,c='.8')
    #dax.plot(ds.lon,0*ds.y,'--',c='.5',lw=.5)
    dax.plot(ds.lon,ds.surface-ds.thickness)
    dax.fill_between(ds.lon[:,0],-1200+0*ds.y,ds.bed[:,0],color='.7',zorder=10)
    dax.fill_between(ds.lon[:,0],ds.bed[:,0],(ds.surface-ds.thickness)[:,0],color='tab:blue')
    dax.fill_between(ds.lon[:,0],ds.surface[:,0],800+0*ds.y,color='skyblue')

    #ds2 = xr.open_dataset(f'../../results/spinup/CrossDots_0.5_mitgcm_{startyear}_{endyear}_100.nc')
    ds2 = xr.open_dataset(f'../../results/CrossDots_0.5_tanh_Tdeep0.4_ztcl-500_050.nc')

    xx2 = ds.x.values[0]
    y02 = ds.y.values[0]
    y12 = ds.y.values[-1]
    ds2 = ds2.sel(x=slice(xx2,xx2),y=slice(y12,y02))

    ds3 = xr.open_dataset('../../../data/gourmelen/CrossonDotson.nc')
    ds3 = add_lonlat(ds3)
    ds3 = ds3.sel(x=slice(xx2,xx2+500),y=slice(y12,y02+500))

    dax = ax[1,aa]
    dax.plot(ds2.lon,ds2.melt,label='LADDIE')
    dax.plot(ds3.lon,ds3.Band1,label='Observations')
    #dax.legend()

for a in [0,1]:
    ax[a,0].axvline(-113.4,0,1,c='.5')
    ax[a,0].axvline(-113.1,0,1,c='k')
    ax[a,1].axvline(-113.22,0,1,c='.5')
    ax[a,1].axvline(-113.0,0,1,c='k')
    ax[a,2].axvline(-113.18,0,1,c='.5')
    ax[a,2].axvline(-113.0,0,1,c='k')

ax[0,0].axvline(-113.4,0,1,c='.5',label='Coastal Ch.')
ax[0,0].axvline(-113.1,0,1,c='k',label='Dotson Ch.')

ax[0,0].set_ylabel('Depth [m]')
ax[1,0].set_ylabel(r'Melt rate [m yr$^{-1}$')
ax[0,0].set_xlim([-113.5,-112.3])
ax[0,0].set_ylim([-500,100])
ax[1,0].set_ylim([0,40])
ax[1,1].set_xlabel(r"Longitude [$^{\circ}$E]")

ax[0,0].set_title('Section A')
ax[0,1].set_title('Section B')
ax[0,2].set_title('Section C')

ax[0,0].legend()
ax[1,0].legend()

plt.savefig('../../figures/draftplot_CD_DC_section.png',dpi=450)
